In [1]:
import glob
import os
import re
import pandas as pd
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from datetime import timedelta
import sys
import datetime
import tensorflow as tf
import math

In [24]:
DATA_PATH = '/kaggle/dev/data-science-bowl-2017-data/stage1_processed/'
OUTPUT_PATH = '/kaggle/dev/data-science-bowl-2017-data/stage1_features_v3/'
PATIENT_SCANS = 'scan_segmented_lungs_fill_'
TENSORBOARD_SUMMARIES = '/kaggle/dev/data-science-bowl-2017-data/tensorboard_summaries/'
MODEL_PATH = '/kaggle_2/luna/luna16/models/9067c9d5-5247-4f9c-8fe7-760a14435c0a/'
OVERLAP_PERCENTAGE = 0.7

In [26]:
def get_patient_data_chunks(patient_id):
    scans = np.load(DATA_PATH + PATIENT_SCANS + patient_id + '.npy')
    chunk_counter = 1
    step_size = int(48 * (1 - OVERLAP_PERCENTAGE))
    num_chunks_0 = int(scans.shape[0] / step_size) + 1
    num_chunks_1 = int(scans.shape[1] / step_size) + 1
    num_chunks_2 = int(scans.shape[2] / step_size) + 1
    X = num_chunks_0 * num_chunks_1 *num_chunks_2
    return X

In [29]:
del chunk_count
del chunks
chunk_count = []
for folder in tqdm(glob.glob(DATA_PATH + PATIENT_SCANS + '*')):                                                                                                     ### TIME STATEMENTS
    m = re.match(PATIENT_SCANS +'([a-f0-9].*).npy', os.path.basename(folder))
    patient_uid = m.group(1)
    
    chunks = get_patient_data_chunks(patient_uid)
    chunk_count.append(chunks)
    tqdm.write("chunks for patient: "+str(patient_uid)  +" : " + str(chunks) )


               | 0/1595 [00:00<?, ?it/s]
  1%|          | 19/1595 [00:51<1:11:28,  2.72s/it]
               | 0/1595 [00:00<?, ?it/s]
  1%|          | 19/1595 [00:51<1:11:30,  2.72s/it]
               | 0/1595 [00:00<?, ?it/s]
  1%|          | 19/1595 [00:51<1:11:32,  2.72s/it]
               | 0/1595 [00:00<?, ?it/s]
  1%|          | 19/1595 [00:51<1:11:34,  2.72s/it]
               | 0/1595 [00:00<?, ?it/s]
  0%|          | 0/1595 [00:00<?, ?it/s]
               | 5/1595 [00:00<00:39, 40.20it/s]
  1%|          | 19/1595 [00:51<1:11:38,  2.73s/it]
               | 5/1595 [00:00<00:47, 33.33it/s]
  0%|          | 5/1595 [00:00<00:57, 27.73it/s]

chunks for patient: 0a0c32c9e08cc2ea76a71649de56be6d : 15000
chunks for patient: 7577cbd6961b0cab27f88727dcd2d6d3 : 9702
chunks for patient: 28352e12fe29361dfd9613ed2e729192 : 18816
chunks for patient: 7852cb521d7029ca08133476054e7bec : 15000
chunks for patient: d5c43054ba0f66d5017a8ddfde8c8c34 : 15548
chunks for patient: 174c5f7c33ca31443208ef873b9477e5 : 15625
chunks for patient: d43c9dd1be361b9302c9343af09cc23e : 18225
chunks for patient: bc43e8a2cb05a45e73dea8c7e02f2cc1 : 16900


          
  1%|          | 19/1595 [00:51<1:11:43,  2.73s/it]
               | 5/1595 [00:00<01:05, 24.32it/s]
  0%|          | 5/1595 [00:00<01:12, 21.89it/s]
               | 9/1595 [00:00<00:39, 39.65it/s]
  1%|          | 19/1595 [00:51<1:11:46,  2.73s/it]
               | 9/1595 [00:00<00:43, 36.61it/s]
  1%|          | 19/1595 [00:51<1:11:48,  2.73s/it]
               | 9/1595 [00:00<00:47, 33.18it/s]
  1%|          | 19/1595 [00:51<1:11:50,  2.74s/it]
               | 9/1595 [00:00<00:51, 30.55it/s]
  1%|          | 19/1595 [00:51<1:11:52,  2.74s/it]
               | 9/1595 [00:00<00:56, 28.26it/s]
  1%|          | 9/1595 [00:00<01:01, 25.93it/s]
               | 14/1595 [00:00<00:39, 40.36it/s]
  1%|          | 19/1595 [00:52<1:11:56,  2.74s/it]
               | 14/1595 [00:00<00:41, 38.06it/s]

chunks for patient: e188bdeea72bb41d980dc2556dc8aafa : 12696
chunks for patient: 49433c1588cc078b825a0eff1dc2e816 : 11132
chunks for patient: 1acbe17dc8f9f59d2fd167b2aa6c650f : 17496
chunks for patient: a19a122fe9a790576b57c6bd5cf9ff5c : 13248
chunks for patient: 6fd582d25eeb2250c2b0996c4216deb9 : 15548
chunks for patient: b1091b5af8b47f24c4b94a9d27c813d1 : 16820
chunks for patient: bdb59c956d48bc1a263acf997926c4d3 : 13125
chunks for patient: 6f229187fe608c9eacc567eb74c1458c : 22869



  1%|          | 19/1595 [00:52<1:11:59,  2.74s/it]
               | 14/1595 [00:00<00:45, 34.44it/s]
  1%|          | 19/1595 [00:52<1:12:02,  2.74s/it]
               | 14/1595 [00:00<00:49, 32.03it/s]
  1%|          | 14/1595 [00:00<00:51, 30.55it/s]
               | 18/1595 [00:00<00:40, 38.94it/s]
  1%|          | 19/1595 [00:52<1:12:05,  2.74s/it]
               | 18/1595 [00:00<00:41, 37.80it/s]
  1%|          | 18/1595 [00:00<00:44, 35.67it/s]

...

chunks for patient: f5713e8c98ef5378ea678fb8d9192e13 : 16224


100%|█████████▉| 1594/1595 [51:06<00:02,  2.11s/it]

chunks for patient: 8ed68f2dbf103a4bc0fd8708d8c1ac93 : 17576


100%|██████████| 1595/1595 [51:08<00:00,  2.15s/it]

chunks for patient: 8918c484841c5d0a532fe146e9da61bd : 17576


In [30]:
len(chunk_count)

1595

In [33]:
total_chunks = np.sum(chunk_count)

In [35]:
443 * total_chunks

10190805668